In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))        

# Any results you write to the current directory are saved as output.

/kaggle/input/av-wns-2019/data_wns/test.csv
/kaggle/input/av-wns-2019/data_wns/sample_submission.csv
/kaggle/input/av-wns-2019/data_wns/train_NA17Sgz/train.csv
/kaggle/input/av-wns-2019/data_wns/train_NA17Sgz/view_log.csv
/kaggle/input/av-wns-2019/data_wns/train_NA17Sgz/item_data.csv


In [2]:
train=pd.read_csv('/kaggle/input/av-wns-2019/data_wns/train_NA17Sgz/train.csv')
test=pd.read_csv('/kaggle/input/av-wns-2019/data_wns/test.csv')
view=pd.read_csv('/kaggle/input/av-wns-2019/data_wns/train_NA17Sgz/view_log.csv')
item=pd.read_csv('/kaggle/input/av-wns-2019/data_wns/train_NA17Sgz/item_data.csv')
s=pd.read_csv('/kaggle/input/av-wns-2019/data_wns/sample_submission.csv')

In [3]:
test.shape

(90675, 6)

In [4]:
train.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0


In [5]:
train.isnull().sum()

impression_id      0
impression_time    0
user_id            0
app_code           0
os_version         0
is_4G              0
is_click           0
dtype: int64

In [6]:
train.user_id.nunique(),test.user_id.nunique()

(74723, 34079)

In [7]:
len(np.intersect1d(train.user_id,view.user_id))
print(len(np.intersect1d(test.user_id,view.user_id)))
print(len(np.intersect1d(test.user_id,train.user_id)))

34079
19645


In [8]:
print(view.shape)
view.head()

(3118622, 5)


,server_time,device_type,session_id,user_id,item_id
0,2018-10-15 08:58:00,android,112333,4557,32970
1,2018-10-15 08:58:00,android,503590,74788,7640
2,2018-10-15 08:58:00,android,573960,23628,128855
3,2018-10-15 08:58:00,android,121691,2430,12774
4,2018-10-15 08:58:00,android,218564,19227,28296


In [9]:
view.isnull().sum()

server_time    0
device_type    0
session_id     0
user_id        0
item_id        0
dtype: int64

In [10]:
item.isnull().sum()

item_id         0
item_price      0
category_1      0
category_2      0
category_3      0
product_type    0
dtype: int64

In [11]:
for k in item.columns:
    print(k,item[k].nunique())
    
item.head()

item_id 132761
item_price 13531
category_1 17
category_2 79
category_3 335
product_type 7959


,item_id,item_price,category_1,category_2,category_3,product_type
0,26880,4602,11,35,20,3040
1,54939,3513,12,57,85,6822
2,40383,825,17,8,279,1619
3,8777,2355,13,58,189,5264
4,113705,1267,17,39,151,10239


In [12]:
item=pd.get_dummies(item,columns=['category_1','category_2'],drop_first=True)

In [13]:
view_item=view.merge(item,on='item_id',how='left')
view_item.head()

,server_time,device_type,session_id,user_id,item_id,item_price,category_3,product_type,category_1_1,category_1_2,...,category_2_70,category_2_71,category_2_72,category_2_73,category_2_74,category_2_75,category_2_76,category_2_77,category_2_78,category_2_79
0,2018-10-15 08:58:00,android,112333,4557,32970,54685.0,253.0,3184.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-10-15 08:58:00,android,503590,74788,7640,1376.0,228.0,545.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2018-10-15 08:58:00,android,573960,23628,128855,4544.0,62.0,5609.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-10-15 08:58:00,android,121691,2430,12774,904.0,252.0,2740.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2018-10-15 08:58:00,android,218564,19227,28296,2304.0,54.0,7422.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
view_item.columns

Index(['server_time', 'device_type', 'session_id', 'user_id', 'item_id',
       'item_price', 'category_3', 'product_type', 'category_1_1',
       'category_1_2',
       ...
       'category_2_70', 'category_2_71', 'category_2_72', 'category_2_73',
       'category_2_74', 'category_2_75', 'category_2_76', 'category_2_77',
       'category_2_78', 'category_2_79'],
      dtype='object', length=102)

In [16]:
view_item['server_time']=pd.to_datetime(view_item['server_time'],format='%Y-%m-%d %H:%M:%S')

view_item.sort_values(['user_id',"server_time"],ascending=True,inplace=True)
view_item['cumcount_1']=view_item.groupby("user_id")["session_id"].cumcount() + 1

view_item.sort_values(['user_id','item_id',"server_time"],ascending=True,inplace=True)
view_item['cumcount_2']=view_item.groupby(["user_id",'item_id'])["session_id"].cumcount() + 1

view_item.sort_values(['user_id','session_id','item_id',"server_time"],ascending=True,inplace=True)
view_item['cumcount_3']=view_item.groupby(["user_id",'session_id','item_id'])["session_id"].cumcount() + 1

view_item['device_type']=view_item['device_type'].astype('category')
view_item['session_id']=view_item['session_id'].astype('category')
view_item['item_id']=view_item['item_id'].astype('category')
view_item['device_type']=view_item['device_type'].astype('category')

view_item.drop(['server_time'],axis=1,inplace=True)

In [17]:
view_item.columns

Index(['device_type', 'session_id', 'user_id', 'item_id', 'item_price',
       'category_3', 'product_type', 'category_1_1', 'category_1_2',
       'category_1_4',
       ...
       'category_2_73', 'category_2_74', 'category_2_75', 'category_2_76',
       'category_2_77', 'category_2_78', 'category_2_79', 'cumcount_1',
       'cumcount_2', 'cumcount_3'],
      dtype='object', length=104)

In [18]:
view_item.head()

,device_type,session_id,user_id,item_id,item_price,category_3,product_type,category_1_1,category_1_2,category_1_4,...,category_2_73,category_2_74,category_2_75,category_2_76,category_2_77,category_2_78,category_2_79,cumcount_1,cumcount_2,cumcount_3
2627295,android,86373,0,116073,4181.0,151.0,124.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33,4,1
1355387,android,143955,0,2715,1601.0,159.0,7093.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,1,1
1355488,android,143955,0,2715,1601.0,159.0,7093.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19,2,2
1355056,android,143955,0,10379,1238.0,272.0,60.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,1,1
1355084,android,143955,0,10379,1238.0,272.0,60.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13,2,2


In [19]:
view_item[view_item.user_id==23628].head()

,device_type,session_id,user_id,item_id,item_price,category_3,product_type,category_1_1,category_1_2,category_1_4,...,category_2_73,category_2_74,category_2_75,category_2_76,category_2_77,category_2_78,category_2_79,cumcount_1,cumcount_2,cumcount_3
1591211,android,67746,23628,1403,441.0,170.0,734.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33,2,1
1591213,android,67746,23628,1403,441.0,170.0,734.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34,3,2
19692,android,79533,23628,86127,2067.0,62.0,5609.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,1,1
19814,android,79533,23628,86127,2067.0,62.0,5609.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,2,2
20124,android,79533,23628,86127,2067.0,62.0,5609.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,3,3


In [20]:
view_item.columns

Index(['device_type', 'session_id', 'user_id', 'item_id', 'item_price',
       'category_3', 'product_type', 'category_1_1', 'category_1_2',
       'category_1_4',
       ...
       'category_2_73', 'category_2_74', 'category_2_75', 'category_2_76',
       'category_2_77', 'category_2_78', 'category_2_79', 'cumcount_1',
       'cumcount_2', 'cumcount_3'],
      dtype='object', length=104)

In [21]:
cat_agg=['count','nunique']
num_agg=['min','mean','max','sum']
agg_col={
    'device_type':cat_agg, 'session_id':cat_agg, 'item_id':cat_agg,'item_price':num_agg,
       'category_3':['count','nunique','mean'], 'product_type':['count','nunique','mean']
}

for k in view_item.columns:
    if k.startswith('category_1') or k.startswith('category_2'):
        agg_col[k]=['sum','mean']
    elif k.startswith('server'):
        agg_col[k]=cat_agg
    elif k.startswith('cumcount'):
        agg_col[k]=num_agg
agg_col

{'device_type': ['count', 'nunique'],
 'session_id': ['count', 'nunique'],
 'item_id': ['count', 'nunique'],
 'item_price': ['min', 'mean', 'max', 'sum'],
 'category_3': ['count', 'nunique', 'mean'],
 'product_type': ['count', 'nunique', 'mean'],
 'category_1_1': ['sum', 'mean'],
 'category_1_2': ['sum', 'mean'],
 'category_1_4': ['sum', 'mean'],
 'category_1_5': ['sum', 'mean'],
 'category_1_6': ['sum', 'mean'],
 'category_1_7': ['sum', 'mean'],
 'category_1_8': ['sum', 'mean'],
 'category_1_9': ['sum', 'mean'],
 'category_1_10': ['sum', 'mean'],
 'category_1_11': ['sum', 'mean'],
 'category_1_12': ['sum', 'mean'],
 'category_1_13': ['sum', 'mean'],
 'category_1_14': ['sum', 'mean'],
 'category_1_15': ['sum', 'mean'],
 'category_1_16': ['sum', 'mean'],
 'category_1_17': ['sum', 'mean'],
 'category_2_1': ['sum', 'mean'],
 'category_2_2': ['sum', 'mean'],
 'category_2_3': ['sum', 'mean'],
 'category_2_4': ['sum', 'mean'],
 'category_2_5': ['sum', 'mean'],
 'category_2_6': ['sum', 'mean'

In [22]:
view_item1=view_item.groupby('user_id').agg(agg_col)

In [23]:
view_item1.head()

device_type         session_id         item_id         item_price  \
              count nunique      count nunique   count nunique        min   
user_id                                                                     
0                42       1         42      11      42      18      332.0   
1                 8       1          8       3       8       8      383.0   
2               165       1        165      37     165     130      128.0   
3                 8       1          8       1       8       3      537.0   
4                 2       1          2       1       2       2     1977.0   

                                            ... cumcount_1        cumcount_2  \
                 mean       max        sum  ...        max    sum        min   
user_id                                     ...                                
0         9395.666667   92160.0   394618.0  ...         42    903          1   
1         3946.750000   12595.0    31574.0  ...          8     36          1   
2        14809.509091  281536.0  2443569.0  ...        165  13695          1   
3         7257.375000   16640.0    58059.0  ...          8     36          1   
4        30114.500000   58252.0    60229.0  ...          2      3          1   

                           cumcount_3                     
             mean max  sum        min      mean max  sum  
user_id                                                   
0        2.190476   6   92          1  1.690476   6   71  
1        1.000000   1    8          1  1.000000   1    8  
2        1.333333   6  220          1  1.072727   4  177  
3        1.875000   3   15          1  1.875000   3   15  
4        1.000000   1    2          1  1.000000   1    2  

[5 rows x 216 columns]

In [24]:
view_item1.columns=['J_' + '_'.join(col).strip() for col in view_item1.columns.values]
view_item1.reset_index(inplace=True)
view_item1.head()

,user_id,J_device_type_count,J_device_type_nunique,J_session_id_count,J_session_id_nunique,J_item_id_count,J_item_id_nunique,J_item_price_min,J_item_price_mean,J_item_price_max,...,J_cumcount_1_max,J_cumcount_1_sum,J_cumcount_2_min,J_cumcount_2_mean,J_cumcount_2_max,J_cumcount_2_sum,J_cumcount_3_min,J_cumcount_3_mean,J_cumcount_3_max,J_cumcount_3_sum
0,0,42,1,42,11,42,18,332.0,9395.666667,92160.0,...,42,903,1,2.190476,6,92,1,1.690476,6,71
1,1,8,1,8,3,8,8,383.0,3946.750000,12595.0,...,8,36,1,1.000000,1,8,1,1.000000,1,8
2,2,165,1,165,37,165,130,128.0,14809.509091,281536.0,...,165,13695,1,1.333333,6,220,1,1.072727,4,177
3,3,8,1,8,1,8,3,537.0,7257.375000,16640.0,...,8,36,1,1.875000,3,15,1,1.875000,3,15
4,4,2,1,2,1,2,2,1977.0,30114.500000,58252.0,...,2,3,1,1.000000,1,2,1,1.000000,1,2


In [25]:
item.item_id.nunique(),view_item1.shape

(132761, (89157, 217))

In [26]:
del view_item
import gc 
gc.collect()

67

In [27]:
s.head()

,impression_id,is_click
0,a9e7126a585a69a32bc7414e9d0c0ada,0
1,caac14a5bf2ba283db7708bb34855760,0
2,13f10ba306a19ce7bec2f3cae507b698,0
3,39c4b4dc0e9701b55a0a4f072008fb3f,0
4,bf5a572cca75f5fc67f4b14e58b11d70,0


In [28]:
df=train.append(test,ignore_index=True)
df.shape
df.head()

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:7116: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,app_code,impression_id,impression_time,is_4G,is_click,os_version,user_id
0,422,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,0,0.0,old,87862
1,467,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,1,1.0,latest,63410
2,259,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,1,0.0,intermediate,71748
3,244,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,1,0.0,latest,69209
4,473,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,0,0.0,latest,62873


In [29]:
df.head()

,app_code,impression_id,impression_time,is_4G,is_click,os_version,user_id
0,422,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,0,0.0,old,87862
1,467,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,1,1.0,latest,63410
2,259,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,1,0.0,intermediate,71748
3,244,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,1,0.0,latest,69209
4,473,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,0,0.0,latest,62873


In [30]:
df=pd.get_dummies(df,columns=['os_version'],drop_first=True)

In [31]:
#preprocessing for df
df['impression_time']=pd.to_datetime(df['impression_time'],format='%Y-%m-%d %H:%M:%S')
df['is_4G']=df['is_4G'].astype('category')
df['app_code']=df['app_code'].astype('category')

df.drop(['impression_time'],axis=1,inplace=True)

df.head()

,app_code,impression_id,is_4G,is_click,user_id,os_version_latest,os_version_old
0,422,c4ca4238a0b923820dcc509a6f75849b,0,0.0,87862,0,1
1,467,45c48cce2e2d7fbdea1afc51c7c6ad26,1,1.0,63410,1,0
2,259,70efdf2ec9b086079795c442636b55fb,1,0.0,71748,0,0
3,244,8e296a067a37563370ded05f5a3bf3ec,1,0.0,69209,1,0
4,473,182be0c5cdcd5072bb1864cdee4d3d6e,0,0.0,62873,1,0


In [32]:
df=df.merge(view_item1,on='user_id',how='left')

df.shape
df.head()

,app_code,impression_id,is_4G,is_click,user_id,os_version_latest,os_version_old,J_device_type_count,J_device_type_nunique,J_session_id_count,...,J_cumcount_1_max,J_cumcount_1_sum,J_cumcount_2_min,J_cumcount_2_mean,J_cumcount_2_max,J_cumcount_2_sum,J_cumcount_3_min,J_cumcount_3_mean,J_cumcount_3_max,J_cumcount_3_sum
0,422,c4ca4238a0b923820dcc509a6f75849b,0,0.0,87862,0,1,1,1,1,...,1,1,1,1.000000,1,1,1,1.000000,1,1
1,467,45c48cce2e2d7fbdea1afc51c7c6ad26,1,1.0,63410,1,0,12,1,12,...,12,78,1,1.916667,5,23,1,1.166667,2,14
2,259,70efdf2ec9b086079795c442636b55fb,1,0.0,71748,0,0,2,1,2,...,2,3,1,1.000000,1,2,1,1.000000,1,2
3,244,8e296a067a37563370ded05f5a3bf3ec,1,0.0,69209,1,0,18,1,18,...,18,171,1,1.388889,3,25,1,1.277778,3,23
4,473,182be0c5cdcd5072bb1864cdee4d3d6e,0,0.0,62873,1,0,46,1,46,...,46,1081,1,1.456522,5,67,1,1.130435,2,52


In [33]:
df['user_id']=df['user_id'].astype('category')
df['is_4G']=df['is_4G'].astype('category')

In [34]:
df.isnull().sum()

app_code                 0
impression_id            0
is_4G                    0
is_click             90675
user_id                  0
                     ...  
J_cumcount_2_sum         0
J_cumcount_3_min         0
J_cumcount_3_mean        0
J_cumcount_3_max         0
J_cumcount_3_sum         0
Length: 223, dtype: int64

In [35]:
df_train=df[df['is_click'].isnull()==False].copy()
df_test=df[df['is_click'].isnull()==True].copy()

print(df_train.shape,df_test.shape)

(237609, 223) (90675, 223)


In [37]:
# del df
gc.collect()

df_train.head()

,app_code,impression_id,is_4G,is_click,user_id,os_version_latest,os_version_old,J_device_type_count,J_device_type_nunique,J_session_id_count,...,J_cumcount_1_max,J_cumcount_1_sum,J_cumcount_2_min,J_cumcount_2_mean,J_cumcount_2_max,J_cumcount_2_sum,J_cumcount_3_min,J_cumcount_3_mean,J_cumcount_3_max,J_cumcount_3_sum
0,422,c4ca4238a0b923820dcc509a6f75849b,0,0.0,87862,0,1,1,1,1,...,1,1,1,1.000000,1,1,1,1.000000,1,1
1,467,45c48cce2e2d7fbdea1afc51c7c6ad26,1,1.0,63410,1,0,12,1,12,...,12,78,1,1.916667,5,23,1,1.166667,2,14
2,259,70efdf2ec9b086079795c442636b55fb,1,0.0,71748,0,0,2,1,2,...,2,3,1,1.000000,1,2,1,1.000000,1,2
3,244,8e296a067a37563370ded05f5a3bf3ec,1,0.0,69209,1,0,18,1,18,...,18,171,1,1.388889,3,25,1,1.277778,3,23
4,473,182be0c5cdcd5072bb1864cdee4d3d6e,0,0.0,62873,1,0,46,1,46,...,46,1081,1,1.456522,5,67,1,1.130435,2,52


In [40]:
df_train=df_train.merge(df_train.groupby(['user_id']).count()[['impression_id']].reset_index().rename(columns={'impression_id':'user_count'}),on='user_id',how='left')
df_test=df_test.merge(df_test.groupby(['user_id']).count()[['impression_id']].reset_index().rename(columns={'impression_id':'user_count'}),on='user_id',how='left')

In [41]:
from catboost import CatBoostClassifier,Pool, cv
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score

In [42]:
X,y=df_train.drop(['impression_id','is_click'],axis=1),df_train['is_click']
Xtest=df_test.drop(['impression_id','is_click'],axis=1)

print(X.shape,Xtest.shape)
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.25,random_state = 1994,stratify=y)

(237609, 222) (90675, 222)


In [43]:
train['is_click'].value_counts()/train.shape[0]

0    0.954286
1    0.045714
Name: is_click, dtype: float64

In [46]:
# Xtest.head()
X.isnull().sum()

app_code             0
is_4G                0
user_id              0
os_version_latest    0
os_version_old       0
                    ..
J_cumcount_3_min     0
J_cumcount_3_mean    0
J_cumcount_3_max     0
J_cumcount_3_sum     0
user_count           0
Length: 222, dtype: int64

In [49]:
err=[]
y_pred_tot=[]
from sklearn.model_selection import KFold,StratifiedKFold
fold=StratifiedKFold(n_splits=10,shuffle=True,random_state=1994)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m=LGBMClassifier(n_estimators=3000,random_state=1994,learning_rate=0.03,colsample_bytree=0.2,objective='binary',scale_pos_weight=1)
    m.fit(X_train,y_train,eval_set=[(X_test, y_test)],eval_metric='auc', early_stopping_rounds=200,verbose=200)
    preds=m.predict_proba(X_test)[:,-1]
    print("err: ",roc_auc_score(y_test,preds))
    err.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(Xtest)[:,-1]
    i=i+1
    y_pred_tot.append(p)

Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_logloss: 0.168698	valid_0's auc: 0.746438
[400]	valid_0's binary_logloss: 0.167574	valid_0's auc: 0.749888
[600]	valid_0's binary_logloss: 0.167415	valid_0's auc: 0.750645
Early stopping, best iteration is:
[525]	valid_0's binary_logloss: 0.167256	valid_0's auc: 0.751311
err:  0.7513112264925058
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_logloss: 0.170478	valid_0's auc: 0.735446
[400]	valid_0's binary_logloss: 0.169739	valid_0's auc: 0.738919
[600]	valid_0's binary_logloss: 0.169722	valid_0's auc: 0.740304
Early stopping, best iteration is:
[528]	valid_0's binary_logloss: 0.169609	valid_0's auc: 0.740465
err:  0.7404650733485545
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_logloss: 0.168564	valid_0's auc: 0.745053
[400]	valid_0's binary_logloss: 0.167726	valid_0's auc: 0.747611
[600]	valid_0's binary_logloss: 0.167639	

In [51]:
np.mean(err,0)

0.7492174421715305

In [52]:
s['is_click']=np.mean(y_pred_tot,0)
s.head()

,impression_id,is_click
0,a9e7126a585a69a32bc7414e9d0c0ada,0.013531
1,caac14a5bf2ba283db7708bb34855760,0.010222
2,13f10ba306a19ce7bec2f3cae507b698,0.062582
3,39c4b4dc0e9701b55a0a4f072008fb3f,0.014803
4,bf5a572cca75f5fc67f4b14e58b11d70,0.190164


In [53]:
s.to_csv('AV_WNS_forkkv2_lgb_folds.csv',index=False)
s.shape

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

(90675, 2)

In [54]:
print(X.shape,Xtest.shape)
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.25,random_state = 1994,stratify=y)
categorical_features_indices = np.where(X_train.dtypes =='category')[0]
categorical_features_indices

(237609, 222) (90675, 222)


array([0, 1, 2])

In [55]:
m=CatBoostClassifier(n_estimators=2500,random_state=1994,learning_rate=0.03,eval_metric='AUC')
m.fit(X_train,y_train,eval_set=[(X_val, y_val.values)], early_stopping_rounds=200,verbose=200,cat_features=categorical_features_indices)
p=m.predict_proba(X_val)[:,-1]
print(roc_auc_score(y_val,p))

0:	test: 0.5418666	best: 0.5418666 (0)	total: 370ms	remaining: 15m 23s
200:	test: 0.7364161	best: 0.7364161 (200)	total: 1m	remaining: 11m 29s
400:	test: 0.7390786	best: 0.7390786 (400)	total: 2m	remaining: 10m 30s
600:	test: 0.7398497	best: 0.7399070 (589)	total: 2m 58s	remaining: 9m 25s
800:	test: 0.7405001	best: 0.7405083 (799)	total: 3m 57s	remaining: 8m 24s
1000:	test: 0.7414331	best: 0.7414480 (995)	total: 4m 57s	remaining: 7m 25s
1200:	test: 0.7416168	best: 0.7416217 (1194)	total: 5m 56s	remaining: 6m 25s
1400:	test: 0.7415737	best: 0.7416946 (1277)	total: 6m 55s	remaining: 5m 25s
1600:	test: 0.7416015	best: 0.7417861 (1448)	total: 7m 55s	remaining: 4m 26s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7417861135
bestIteration = 1448

Shrink model to first 1449 iterations.
0.7417861135403558


In [58]:
errCB=[]
y_pred_tot_cb=[]
from sklearn.model_selection import KFold,StratifiedKFold
fold=StratifiedKFold(n_splits=10,shuffle=True,random_state=1994)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m=CatBoostClassifier(n_estimators=2500,random_state=1994,eval_metric='AUC',learning_rate=0.03)
    m.fit(X_train,y_train,eval_set=[(X_test, y_test)], early_stopping_rounds=200,verbose=200,cat_features=categorical_features_indices)
    preds=m.predict_proba(X_test)[:,-1]
    print("err_cb: ",roc_auc_score(y_test,preds))
    errCB.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(Xtest)[:,-1]
    i=i+1
    y_pred_tot_cb.append(p)

0:	test: 0.5144795	best: 0.5144795 (0)	total: 332ms	remaining: 13m 49s
200:	test: 0.7436090	best: 0.7436090 (200)	total: 1m 7s	remaining: 12m 47s
400:	test: 0.7464037	best: 0.7464189 (397)	total: 2m 14s	remaining: 11m 42s
600:	test: 0.7471002	best: 0.7471002 (600)	total: 3m 19s	remaining: 10m 29s
800:	test: 0.7484396	best: 0.7484861 (797)	total: 4m 24s	remaining: 9m 21s
1000:	test: 0.7489092	best: 0.7490326 (885)	total: 5m 30s	remaining: 8m 14s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7490325983
bestIteration = 885

Shrink model to first 886 iterations.
err_cb:  0.7490325983432549
0:	test: 0.5255802	best: 0.5255802 (0)	total: 353ms	remaining: 14m 41s
200:	test: 0.7349605	best: 0.7349727 (196)	total: 1m 11s	remaining: 13m 41s
400:	test: 0.7367486	best: 0.7367954 (398)	total: 2m 22s	remaining: 12m 27s
600:	test: 0.7380834	best: 0.7381583 (588)	total: 3m 31s	remaining: 11m 9s
800:	test: 0.7385614	best: 0.7385799 (798)	total: 4m 41s	remaining: 9m 56s
1000:	test:

In [59]:
np.mean(errCB,0)

0.7484512189560181

In [60]:
s['is_click']=np.mean(y_pred_tot_cb,0)
s.head()

,impression_id,is_click
0,a9e7126a585a69a32bc7414e9d0c0ada,0.016591
1,caac14a5bf2ba283db7708bb34855760,0.031968
2,13f10ba306a19ce7bec2f3cae507b698,0.077346
3,39c4b4dc0e9701b55a0a4f072008fb3f,0.012104
4,bf5a572cca75f5fc67f4b14e58b11d70,0.141255


In [61]:
s.to_csv('AV_WNS_forkkv2_cb_folds.csv',index=False)
s.shape

(90675, 2)

In [64]:
s['is_click']=np.mean(y_pred_tot_cb,0)*0.65 + np.mean(y_pred_tot,0)*0.35
s.head()

,impression_id,is_click
0,a9e7126a585a69a32bc7414e9d0c0ada,0.015520
1,caac14a5bf2ba283db7708bb34855760,0.024357
2,13f10ba306a19ce7bec2f3cae507b698,0.072179
3,39c4b4dc0e9701b55a0a4f072008fb3f,0.013049
4,bf5a572cca75f5fc67f4b14e58b11d70,0.158373


In [65]:
s.to_csv('AV_WNS_forkkv2_CBstack_folds.csv',index=False)
s.shape

(90675, 2)